Shape data in "77_cancer_proteomes_CPTAC_itraq.csv" from colum mode to raw mode.


The data are the raw exported in CSV format and loaded in a DashDB cloud instance (jdbc:db2://dashdb-entry-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB:retrieveMessagesFromServerOnGetMessage=true)
in  a table with following DDL:

CREATE TABLE "DASH13540"."BREASTCANCER_GENOMES" (
		"RefSeq_accession_number" CHAR(100 OCTETS), 
		"gene_symbol" CHAR(20 OCTETS), 
		"gene_name" CHAR(250 OCTETS), 
		"Patient" CHAR(28 OCTETS),
		"Value" DOUBLE, 
		"Gene" CHAR(250 OCTETS) NOT NULL, 
		"Patient_ID" CHAR(24 OCTETS) NOT NULL
	)
	ORGANIZE BY COLUMN
	DATA CAPTURE NONE;

With following SQL i verify data duplication:

select distinct "Patient_ID"
 from "DASH13540"."BREASTCANCER_GENOMES"
 group by "Gene",
          "Patient_ID"
 having count(*)>1; 
 
I observer that 3 Patient_ID (AO-A12D, C8-A131, A0-A12B) have duplicated column in term of proteomes values having a single status record in Patient data.

Consquently i delete those column generating a new csv input file "77_cancer_proteomes_CPTAC_itraq_NODouble.csv" and again a reshaped it

I also observed that last 3 column in data matrix refer to heatly patient (without record in patient statis data). So i change column in HEALTY1,2,3 and i add those 3 pseudo patient to patient record.

Again i create a new csv file "clinical_data_breast_cancer_withHealty.csv" and i use this

In notebook i maintain alla passage

In [15]:
! pip install --upgrade --user numpy scipy matplotlib ipython jupyter pandas sympy 
! pip install --user --upgrade pixiedust

Requirement already up-to-date: numpy in /home/dsxuser/.local/lib/python3.5/site-packages
Requirement already up-to-date: scipy in /home/dsxuser/.local/lib/python3.5/site-packages
Requirement already up-to-date: matplotlib in /home/dsxuser/.local/lib/python3.5/site-packages
Requirement already up-to-date: ipython in /home/dsxuser/.local/lib/python3.5/site-packages
Requirement already up-to-date: jupyter in /home/dsxuser/.local/lib/python3.5/site-packages
Requirement already up-to-date: pandas in /home/dsxuser/.local/lib/python3.5/site-packages
Requirement already up-to-date: sympy in /home/dsxuser/.local/lib/python3.5/site-packages
Requirement not upgraded as not directly required: kiwisolver>=1.0.1 in /home/dsxuser/.local/lib/python3.5/site-packages (from matplotlib)
Requirement not upgraded as not directly required: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from matplotlib)
Requirement not upgraded as not directly required: 

In [16]:
from io import StringIO
from scipy import signal
from matplotlib import pylab as pl
from mpl_toolkits.mplot3d.axes3d import Axes3D
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pixiedust
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
from project_lib import Project

In [17]:
#The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
project = Project(project_id='8cec06cd-ebbe-4f7a-9f66-8e1e40722f95', project_access_token='p-1298f2f1ff374652829046e40c4f21f2e59886f1')
pc = project.project_context

In [18]:
# The code was removed by Watson Studio for sharing.

,RefSeq_accession_number,gene_symbol,gene_name,AO-A12D.01TCGA,C8-A131.01TCGA,AO-A12B.01TCGA,BH-A18Q.02TCGA,C8-A130.02TCGA,C8-A138.03TCGA,E2-A154.03TCGA,...,AO-A12B.34TCGA,A2-A0SW.35TCGA,AO-A0JL.35TCGA,BH-A0BV.35TCGA,A2-A0YM.36TCGA,BH-A0C7.36TCGA,A2-A0SX.36TCGA,263d3f-I.CPTAC,blcdb9-I.CPTAC,c4155b-C.CPTAC
0,NP_958782,PLEC,plectin isoform 1,1.096131,2.609943,-0.659828,0.195341,-0.494060,2.765081,0.862659,...,-0.963904,-0.487772,-0.10668,-0.065838,0.655850,-0.552212,-0.398560,0.598585,-0.191285,0.566975
1,NP_958785,NaN,plectin isoform 1g,1.111370,2.650422,-0.648742,0.215413,-0.503899,2.779709,0.870186,...,-0.938210,-0.487772,-0.10668,-0.055893,0.658143,-0.547749,-0.392601,0.606697,-0.183918,0.578702
2,NP_958786,PLEC,plectin isoform 1a,1.111370,2.650422,-0.654285,0.215413,-0.500619,2.779709,0.870186,...,-0.943919,-0.487772,-0.10668,-0.065838,0.655850,-0.552212,-0.392601,0.603993,-0.186022,0.576747
3,NP_000436,NaN,plectin isoform 1c,1.107561,2.646374,-0.632113,0.205377,-0.510459,2.797995,0.866423,...,-0.935355,-0.487772,-0.10668,-0.055893,0.655850,-0.552212,-0.392601,0.603993,-0.186022,0.576747
4,NP_958781,NaN,plectin isoform 1e,1.115180,2.646374,-0.640428,0.215413,-0.503899,2.787023,0.870186,...,-0.935355,-0.503853,-0.10668,-0.062523,0.651264,-0.556675,-0.395581,0.603993,-0.167079,0.576747


In [19]:
body = client_8a4fc8dc8bf54832ab62f63af0c03772.get_object(Bucket='breastcancerproteomes-donotdelete-pr-zmcmbnmwpo1wza',Key='clinical_data_breast_cancer.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

patientData = pd.read_csv(body)
patientData.head()

,Complete TCGA ID,Gender,Age at Initial Pathologic Diagnosis,ER Status,PR Status,HER2 Final Status,Tumor,Tumor--T1 Coded,Node,Node-Coded,...,PAM50 mRNA,SigClust Unsupervised mRNA,SigClust Intrinsic mRNA,miRNA Clusters,methylation Clusters,RPPA Clusters,CN Clusters,Integrated Clusters (with PAM50),Integrated Clusters (no exp),Integrated Clusters (unsup exp)
0,TCGA-A2-A0T2,FEMALE,66,Negative,Negative,Negative,T3,T_Other,N3,Positive,...,Basal-like,0,-13,3,5,Basal,3,2,2,2
1,TCGA-A2-A0CM,FEMALE,40,Negative,Negative,Negative,T2,T_Other,N0,Negative,...,Basal-like,-12,-13,4,4,Basal,4,2,1,1
2,TCGA-BH-A18V,FEMALE,48,Negative,Negative,Negative,T2,T_Other,N1,Positive,...,Basal-like,-12,-13,5,5,Basal,1,2,2,2
3,TCGA-BH-A18Q,FEMALE,56,Negative,Negative,Negative,T2,T_Other,N1,Positive,...,Basal-like,-12,-13,5,5,Basal,1,2,2,2
4,TCGA-BH-A0E0,FEMALE,38,Negative,Negative,Negative,T3,T_Other,N3,Positive,...,Basal-like,0,-13,5,5,Basal,1,2,2,2


In [20]:
reshapedData=pd.melt(orgData,id_vars=["RefSeq_accession_number","gene_symbol","gene_name"],var_name="Patient",value_name="Value")
reshapedData.head()

,RefSeq_accession_number,gene_symbol,gene_name,Patient,Value
0,NP_958782,PLEC,plectin isoform 1,AO-A12D.01TCGA,1.096131
1,NP_958785,NaN,plectin isoform 1g,AO-A12D.01TCGA,1.111370
2,NP_958786,PLEC,plectin isoform 1a,AO-A12D.01TCGA,1.111370
3,NP_000436,NaN,plectin isoform 1c,AO-A12D.01TCGA,1.107561
4,NP_958781,NaN,plectin isoform 1e,AO-A12D.01TCGA,1.115180


In [21]:
reshapedData["Gene"]=reshapedData["RefSeq_accession_number"].fillna('#')+"-"+reshapedData["gene_symbol"].fillna('#')+" ("+reshapedData["gene_name"].fillna('#')+")"
reshapedData["Patient_ID"]="TCGA-" + reshapedData["Patient"].str.slice(0,7)
reshapedData.head()

,RefSeq_accession_number,gene_symbol,gene_name,Patient,Value,Gene,Patient_ID
0,NP_958782,PLEC,plectin isoform 1,AO-A12D.01TCGA,1.096131,NP_958782-PLEC (plectin isoform 1),TCGA-AO-A12D
1,NP_958785,NaN,plectin isoform 1g,AO-A12D.01TCGA,1.111370,NP_958785-# (plectin isoform 1g),TCGA-AO-A12D
2,NP_958786,PLEC,plectin isoform 1a,AO-A12D.01TCGA,1.111370,NP_958786-PLEC (plectin isoform 1a),TCGA-AO-A12D
3,NP_000436,NaN,plectin isoform 1c,AO-A12D.01TCGA,1.107561,NP_000436-# (plectin isoform 1c),TCGA-AO-A12D
4,NP_958781,NaN,plectin isoform 1e,AO-A12D.01TCGA,1.115180,NP_958781-# (plectin isoform 1e),TCGA-AO-A12D


In [22]:
#verify what separator to use in out CSV to avoid further db load problem
coma_count=reshapedData['Gene'].str.contains(';').sum()
print (coma_count)

0


In [23]:
project.save_data(data=reshapedData.to_csv(index=False, sep=';', encoding='utf-8'),file_name='Reshaped_data1.csv',overwrite=True)

{'asset_id': '25330465-2ad0-4e1b-8de1-fd154128a0f6',
 'bucket_name': 'breastcancerproteomes-donotdelete-pr-zmcmbnmwpo1wza',
 'file_name': 'Reshaped_data1.csv',
 'message': 'File Reshaped_data1.csv has been written successfully to the associated OS'}

In [24]:
# I reimport cleaned CSV

body = client_8a4fc8dc8bf54832ab62f63af0c03772.get_object(Bucket='breastcancerproteomes-donotdelete-pr-zmcmbnmwpo1wza',Key='77_cancer_proteomes_CPTAC_itraq_NODouble.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

nodoubled_data = pd.read_csv(body)
nodoubled_data.head()

,RefSeq_accession_number,gene_symbol,gene_name,BH-A18Q.02TCGA,C8-A130.02TCGA,C8-A138.03TCGA,E2-A154.03TCGA,C8-A12L.04TCGA,A2-A0EX.04TCGA,AN-A04A.05TCGA,...,AR-A0TT.34TCGA,A2-A0SW.35TCGA,AO-A0JL.35TCGA,BH-A0BV.35TCGA,A2-A0YM.36TCGA,BH-A0C7.36TCGA,A2-A0SX.36TCGA,HEALTY1,HEALTY2,HEALTY3
0,NP_958782,PLEC,plectin isoform 1,0.195341,-0.494060,2.765081,0.862659,1.407570,1.185108,0.384588,...,-0.511421,-0.487772,-0.10668,-0.065838,0.655850,-0.552212,-0.398560,0.598585,-0.191285,0.566975
1,NP_958785,NaN,plectin isoform 1g,0.215413,-0.503899,2.779709,0.870186,1.407570,1.192612,0.371393,...,-0.526067,-0.487772,-0.10668,-0.055893,0.658143,-0.547749,-0.392601,0.606697,-0.183918,0.578702
2,NP_958786,PLEC,plectin isoform 1a,0.215413,-0.500619,2.779709,0.870186,1.410312,1.188860,0.371393,...,-0.526067,-0.487772,-0.10668,-0.065838,0.655850,-0.552212,-0.392601,0.603993,-0.186022,0.576747
3,NP_000436,NaN,plectin isoform 1c,0.205377,-0.510459,2.797995,0.866423,1.407570,1.185108,0.377990,...,-0.533389,-0.487772,-0.10668,-0.055893,0.655850,-0.552212,-0.392601,0.603993,-0.186022,0.576747
4,NP_958781,NaN,plectin isoform 1e,0.215413,-0.503899,2.787023,0.870186,1.413053,1.200116,0.374692,...,-0.529728,-0.503853,-0.10668,-0.062523,0.651264,-0.556675,-0.395581,0.603993,-0.167079,0.576747


In [25]:
reshapedData1=pd.melt(nodoubled_data,id_vars=["RefSeq_accession_number","gene_symbol","gene_name"],var_name="Patient",value_name="Value")
reshapedData1.head()

,RefSeq_accession_number,gene_symbol,gene_name,Patient,Value
0,NP_958782,PLEC,plectin isoform 1,BH-A18Q.02TCGA,0.195341
1,NP_958785,NaN,plectin isoform 1g,BH-A18Q.02TCGA,0.215413
2,NP_958786,PLEC,plectin isoform 1a,BH-A18Q.02TCGA,0.215413
3,NP_000436,NaN,plectin isoform 1c,BH-A18Q.02TCGA,0.205377
4,NP_958781,NaN,plectin isoform 1e,BH-A18Q.02TCGA,0.215413


In [26]:
reshapedData1["Gene"]=reshapedData1["RefSeq_accession_number"].fillna('#')+"-"+reshapedData1["gene_symbol"].fillna('#')+" ("+reshapedData1["gene_name"].fillna('#')+")"
reshapedData1["Patient_ID"]="TCGA-" + reshapedData1["Patient"].str.slice(0,7)
reshapedData1.head()

,RefSeq_accession_number,gene_symbol,gene_name,Patient,Value,Gene,Patient_ID
0,NP_958782,PLEC,plectin isoform 1,BH-A18Q.02TCGA,0.195341,NP_958782-PLEC (plectin isoform 1),TCGA-BH-A18Q
1,NP_958785,NaN,plectin isoform 1g,BH-A18Q.02TCGA,0.215413,NP_958785-# (plectin isoform 1g),TCGA-BH-A18Q
2,NP_958786,PLEC,plectin isoform 1a,BH-A18Q.02TCGA,0.215413,NP_958786-PLEC (plectin isoform 1a),TCGA-BH-A18Q
3,NP_000436,NaN,plectin isoform 1c,BH-A18Q.02TCGA,0.205377,NP_000436-# (plectin isoform 1c),TCGA-BH-A18Q
4,NP_958781,NaN,plectin isoform 1e,BH-A18Q.02TCGA,0.215413,NP_958781-# (plectin isoform 1e),TCGA-BH-A18Q


In [27]:
project.save_data(data=reshapedData1.to_csv(index=False, sep=';', encoding='utf-8'),file_name='Reshaped_data_nodouble.csv',overwrite=True)

{'asset_id': '46644faf-3fe6-4479-9a86-f11cc7cdbefc',
 'bucket_name': 'breastcancerproteomes-donotdelete-pr-zmcmbnmwpo1wza',
 'file_name': 'Reshaped_data_nodouble.csv',
 'message': 'File Reshaped_data_nodouble.csv has been written successfully to the associated OS'}